In [13]:
import rolch
import numpy as np
from sklearn.datasets import load_diabetes

np.set_printoptions(precision=3, suppress=True)

# Batch Estimation

In [14]:
## Diabetes data set
## Add intercept (will not be regularized)

X, y = load_diabetes(return_X_y=True)
X = np.hstack((np.ones((X.shape[0], 1)), X))

## OLS

In [15]:
online_gamlss_ols = rolch.OnlineGamlss(
    distribution=rolch.DistributionT(), 
    method="ols"
)

online_gamlss_ols.fit(y, X, X, X)

print("OLS Coefficients \n")
print(np.vstack(online_gamlss_ols.betas).T)

OLS Coefficients 

[[148.321   3.912   2.925]
 [ -1.694  -0.034   0.035]
 [-10.653  -0.026   0.156]
 [ 16.167   0.066  -0.067]
 [ 12.887   0.05   -0.146]
 [-29.789   0.019   2.24 ]
 [ 18.741  -0.051  -2.019]
 [  1.384   0.     -0.515]
 [  2.927   0.026   0.109]
 [ 28.401  -0.006  -0.818]
 [  1.391   0.04   -0.09 ]]


## LASSO

In [16]:
dist = rolch.DistributionT()

online_gamlss_lasso = rolch.OnlineGamlss(
    distribution=rolch.DistributionT(),
    method="lasso",
    estimation_kwargs={"ic" : {i: "bic" for i in range(dist.n_params)}},
)

online_gamlss_lasso.fit(y, X, X, X)

print("LASSO Coefficients \n")
print(np.vstack(online_gamlss_lasso.betas).T)

LASSO Coefficients 

[[151.477   3.922   9.163]
 [  0.191  -0.045   0.957]
 [-11.589  -0.105   0.015]
 [ 22.3     0.046  -0.237]
 [ 15.728   0.094  -0.171]
 [ -9.854  -0.001  57.339]
 [  0.022   0.078 -50.351]
 [ -7.917  -0.264 -22.77 ]
 [  4.73   -0.184  -0.331]
 [ 26.556   0.053 -21.377]
 [  3.017   0.052  -0.342]]


# Incremental Fit

In [17]:
online_gamlss_lasso = rolch.OnlineGamlss(
    distribution=rolch.DistributionT(), 
    method="ols", 
    estimation_kwargs={"ic" : "bic"}, 
    # aic, bic, hqc, max (max == always select largest model --> OLS solution)
)

online_gamlss_lasso.model_selection_on_weighted_rss = False

online_gamlss_lasso.fit(
    y[:-1], 
    X[:-1, :], 
    X[:-1, :], 
    X[:-1, :]
)

print("Coefficients for the first N-1 observations \n")
print(np.vstack(online_gamlss_lasso.betas).T)

print("\nRSS for the first N-1 observations \n")
print(online_gamlss_lasso.rss)

Coefficients for the first N-1 observations 

[[148.608   3.913   2.925]
 [ -1.715  -0.034   0.035]
 [-10.638  -0.026   0.157]
 [ 16.137   0.066  -0.068]
 [ 12.81    0.05   -0.149]
 [-29.945   0.021   2.236]
 [ 18.908  -0.052  -2.02 ]
 [  1.638  -0.002  -0.502]
 [  3.088   0.025   0.113]
 [ 28.54   -0.007  -0.821]
 [  1.415   0.04   -0.089]]

RSS for the first N-1 observations 

{0: 0.5607622474680065, 1: 0.3138116245131759, 2: 0.16129157285356924}


In [18]:
online_gamlss_lasso.update(
    y[[-1]], 
    X[[-1], :], 
    X[[-1], :], 
    X[[-1], :]
)

print("\nCoefficients after update call \n")
print(np.vstack(online_gamlss_lasso.betas).T)

print("\nRSS after update call  \n")
print(online_gamlss_lasso.rss)


Coefficients after update call 

[[151.881   3.966   4.173]
 [ -0.765  -0.047   0.13 ]
 [-12.968  -0.069   0.4  ]
 [ 23.564   0.084  -0.121]
 [ 15.931   0.094  -0.361]
 [-26.278  -0.463  11.797]
 [ 13.564   0.408 -10.664]
 [ -1.327   0.062  -3.344]
 [  6.064  -0.058   0.455]
 [ 32.744   0.216  -4.371]
 [  2.005   0.064  -0.208]]

RSS after update call  

{0: 0.5607622474680065, 1: array([0.314]), 2: array([0.164])}
